# Project title

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys

In [42]:
#### Pull Information Off Each Listing Stored in URLS List ####

carlist = []

### READ DATA ###
with open("./data/Jaguar_E_Type.html", "r") as file:
    test_data = file.read()
out = test_data.split(delimiter)

#url = 'https://bringatrailer.com/listing/1962-jaguar-xke-roadster-18/'
for i in range(0,len(urls)):
    
    #r = requests.get(urls[i])
    #soup = BeautifulSoup(r.content)
    #print(soup.prettify())
    soup = BeautifulSoup(out[i], 'html.parser')
    #Get Year
    title = str(soup.find('title'))
    loc = str(soup.findAll('a'))
    i = 0

    if '19' in title:
        index = title.index('19')
        year = title[index:index+4]
    else:
        year = 'N/A'
    #while i < len(title):
     #   if title[i:i+4].isdigit() == True:
     #       year = title[i:i+4]
    #      i = len(title)
       # i += 1

        
    #Check if Sold and get highest bid
    if 'sold for' in title:
        sold = 1
        ind = title.index('$')
        spl_word = ' '
        price = title[ind+1:ind+20].partition(spl_word)[0]
    elif 'bid to $' in loc:
        sold = 0
        index = loc.index('bid to $')
        price = loc[index+8:index+15].partition('<')[0]
    else:
        price = 'N/A'



    #Get Transmission
    transmission = str(soup.findAll('li'))
    i = 0
    trans = 0
    if 'Automatic' in transmission:
        trans = 1
    #trans = 'Manual'
    #while i < len(transmission):
     #   if 'Automatic' in transmission[i:i+9]:
      #      trans = 1
       #     #trans = 'Automatic'
        #    i = len(transmission)
        #i += 1


    #Get Location
    
    index = loc.index('/place/')
    town = loc[index+7:index+40].partition(',')[0]
    town = town.replace('%20', ' ')
    
    state = loc[index+7:index+200].partition(',')[2]
    state = state.partition('"')[0]
    state = state.replace('%20', ' ')
    state = re.sub(r'[0-9]+', '', state)

    contents = str(soup.findAll('p'))

    #Check for Rust
    if ' rust' in contents or 'Rust' in contents:
        rust = 1
    else:
        rust = 0

    #Check for Refurbishment
    if 'refurbish' in contents or 'Refurbish' in contents:
        refurbished = 1
    else:
        refurbished = 0

    #Check for Restoration
    if 'restor' in contents or 'Restor' in contents:
        restored = 1
    else:
        restored = 0
    
    #Check for Project
    if 'project' in title or 'Project in title':
        project = 1
    else:
        project = 0
    
    #Get Lot Number
    index = title.index('Lot #')
    lotnumb = title[index+5:index+12].partition(')')[0]
    
    #Get Month
    if 'January' in title:
        month = 1
    elif 'February' in title:
        month = 2
    elif 'March' in title:
        month = 3
    elif 'April' in title:
        month = 4
    elif 'May' in title:
        month = 5
    elif 'June' in title:
        month = 6
    elif 'July' in title:
        month = 7
    elif 'August' in title:
        month = 8
    elif 'September' in title:
        month = 9
    elif 'October' in title:
        month = 10
    elif 'November' in title:
        month = 11
    elif 'December' in title:
        month = 12
    
    #Get Year
    index = title.index('(')
    yearsold = title[index-5:index-1]
    
    ### JAGUAR SPECIFIC ATTRIBUTES ###
    #Get Mark
    engine = '10'
    if 'Series' in title:
        index = title.index('Series')
        test_str = title[index+7:index+10]
        if 'III' in test_str:
            mk = 3
            engine = 5.3
        elif 'II' in test_str:
            mk = 2
        elif 'I' in test_str:
            mk = 1
        else:
            mk = 1.5
    if 'V12' in title or 'V-12' in title:
        mk = 3
        engine = 5.3
    
    #Various other Condition terms
    scratch = 0
    if 'scratch' in contents or'Scratch' in contents:
        scratch = 1
    paintbub = 0
    if 'paint bubble' in contents or 'Paint bubble' in contents:
        paintbub = 1
    metalrepair = 0
    if 'metal repair' in contents or 'Metal repair' in contents:
        metalrepair = 1
    
    #Get Engine and Number Matching
    numbmatch = 0
    
    if 'Numbers-Matching' in transmission:
        numbmatch = 1
        index = transmission.index('Matching')
        engine = transmission[index+8:index+13].partition(' ')[2]
        engine = engine.partition('-')[0]
        engine = engine.partition('L')[0]
        
    elif 'Liter' in transmission:
        index = transmission.index('Liter')
        engine = transmission[index-15:index].partition('>')[2]
        engine = re.sub(r'[^0-9.]', '', engine)

    elif 'Twin Cam' in transmission or 'Twin-Cam' in transmission:
        index = transmission.index('Twin')
        engine = transmission[index-30:index+30]
        if '.' in engine:
            index = engine.index('.')
            engine = engine[index-5:index+5]
        engine = re.sub(r'[^0-9.]', '', engine)
        
    elif 'Inline-Six' in transmission or 'Inline Six' in transmission:
        index = transmission.index('Inline')
        engine = transmission[index-30:index+20]
        if '.' in engine:
            index = engine.index('.')
            engine = engine[index-5:index+5]
        engine = re.sub(r'[^0-9.]', '', engine)
        
    elif 'Straight-Six' in transmission or 'Straight Six' in transmission:
        index = transmission.index('Straight')
        engine = transmission[index-15:index-1].partition('>')[2]
        engine = re.sub(r'[^0-9.]', '', engine)
        
    #Check for Heritage Trust Certficiate
    trustcert = 0
    if 'Jaguar Daimler Heritage Trust' in contents:
        trustcert = 1
    
    ### AUSTIN HEALEY 3000 SPECIFIC ATTRIBUTES ###
    if 'hardtop' in contents or 'Hardtop' in contents:
        hardtop = 1
    else:
        hardtop = 0
    
    if 'overdrive' in contents or 'Overdrive' in contents:
        overdrive = 1
    else:
        overdrive = 0
    
    if '5-speed' in contents or '5-Speed' in contents:
        transswap = 1
    else:
        transswap = 0
    
    #i = 0
    #while i < len(title):
    #    if title[i] == 'B' and title[i+2].isdigit() == True:
    #        accessory = title[i:i+3]
    #        i = len(title)
    #    i += 1
        
        
        
    #Get Mileage
    mileage = str(soup.findAll('li'))
    miles = 'TMU'
    milestmu = 'N/A'
    if 'Miles' in mileage:
        index = mileage.index('Miles')
    elif 'miles' in mileage:
        index = mileage.index('miles')
        
    if 'TMU' not in mileage[index+10:index+20]:
        miles = mileage[index-12:index].partition('<li>')[2]
        if 'k' in miles or 'K' in miles:
            miles = re.sub(r'[^0-9]', '', miles)
            miles = int(miles) * 1000
        else:
            miles = re.sub(r'[^0-9]', '', miles)
    else:
        milestmu = mileage[index-12:index].partition('<li>')[2]
        if 'k' in milestmu or 'K' in milestmu:
            milestmu = re.sub(r'[^0-9]', '', milestmu)
            milestmu = int(milestmu) * 1000
        else:
            milestmu = re.sub(r'[^0-9]', '', milestmu)
            
    
    ### LANCIA SPECIFIC ATTRIBUTES ###
    #Get Mileage
    #mileage = str(soup.findAll('li'))
    #miles = 'TMU'
    #if 'Kilometers' in mileage and 'Indicated' not in mileage:
    #    index = mileage.index('Kilometers')
    #    if 'TMU' not in mileage[index+10:index+20]:
    #        miles = mileage[index-12:index].partition('<li>')[2]
    #       miles = miles.replace(',','')
    #        if 'k' in miles:
    #            miles = miles.replace('k','')
    #            miles = int(miles) * 1000
    #        miles = int(miles)/1.60934
    
    #if 'Zagato' in title:
    #    zagato = 1
    #else:
    #    zagato = 0

    Car = [lotnumb, trans, year, miles, milestmu, engine, numbmatch, hardtop, overdrive, scratch, paintbub, metalrepair, rust, refurbished, restored, project, trustcert, sold, month, yearsold, price, town, state]
    #Car = {'Year': year, 'Mark': mk, 'Mileage': miles, 'Transmission': trans, 'Location': location, 'Rust': rust, 'Refurbished': refurbished, 'Restored': restored, 'Sold': sold, 'Highest Bid': price}
    carlist.append(Car)
    #print(miles)
df = pd.DataFrame(carlist, columns=['Lot Number', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)','Engine', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', 'Rust','Refurbished','Restored', 'Project', 'Jaguar Daimler Heritage Trust Certificate', 'Sold', 'Month', 'Year', 'Price', 'City', 'State'])

display(df)




,Lot Number,Transmission,Model Year,Mileage,Mileage (if TMU),Engine,Numbers Matching,Hardtop,Overdrive,Scratch,...,Refurbished,Restored,Project,Jaguar Daimler Heritage Trust Certificate,Sold,Month,Year,Price,City,State
0,"95,458",0,1961,TMU,82000,3.8,0,0,0,0,...,1,0,1,1,0,1,2023,"205,000",Soest,Netherlands
1,"95,404",0,1964,TMU,4800,10,0,0,0,0,...,1,1,1,0,0,1,2023,"151,000",West Vancouver,"British Columbia, Canada"
2,"95,283",0,1968,TMU,77000,4.2,0,0,0,0,...,1,0,1,0,1,1,2023,"95,000",Hudson,Ohio
3,"95,040",0,1969,TMU,18000,4.2,0,0,0,0,...,1,0,1,0,0,1,2023,"62,000",La Jolla,California
4,"95,002",0,1961,TMU,99000,3.8,0,0,0,0,...,0,0,1,0,0,1,2023,"115,000",Soest,Netherlands
5,"94,796",0,1967,42000,N/A,4.2,0,0,0,0,...,1,0,1,0,1,12,2022,"115,000",West Lancaster,Pennsylvania
6,"94,497",0,1969,TMU,8200,4.2,0,0,0,0,...,1,0,1,0,0,12,2022,"48,000",Hillsboro,Oregon
7,"94,472",0,1970,TMU,62000,4.2,0,0,0,0,...,0,0,1,0,1,12,2022,"65,500",Davenport,Iowa
8,"94,546",0,1964,TMU,40000,3.8,0,0,0,0,...,1,0,1,0,0,12,2022,"80,000",Boca Raton,Florida
9,"94,353",0,1965,TMU,15000,4.2,0,0,0,0,...,1,0,1,1,0,12,2022,"171,000",Scottsdale,Arizona


In [32]:
#### Get URL List ####
#### DO NOT DELETE ####

url = "https://bringatrailer.com/jaguar/xke/"
urls = []
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

url = str(soup.findAll("div"))

i = 0

while i < len(url):
    if url[i : i + 5] == '"url"':
        tempurl = url[i + 7 : i + 200].partition('"')[0]
        if (
            "listing" in tempurl
            and "hardtop" not in tempurl
            and "memorabilia" not in tempurl
            and "tool kit" not in tempurl
        ):
            if "\\" in tempurl:
                tempurl = tempurl.replace("\\", "")
                if tempurl not in urls:
                    urls.append(tempurl.replace("\\", ""))
    i += 1


def get_data(url):

    r = requests.get(url)

    soup = BeautifulSoup(r.content, "html.parser")
    return str(soup)


souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in urls)


In [45]:
### PUSH TO EXCEL ###
df.to_excel('output.xlsx')

ModuleNotFoundError: No module named 'openpyxl'

In [3]:
#### Get List of Every Result ####
#### DO NOT DELETE ####

import requests
from bs4 import BeautifulSoup


urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=1833076%2C1833077%2C44175474&sort=td&page='
urlend = '&results=items'
url = []
urls = []
pgnum = 1
while pgnum < 22:
    url.append(urlstart + str(pgnum) + urlend)
    pgnum += 1

for j in range(0, len(url)):
    print(url[j])
    r = requests.get(url[j])
    soup = BeautifulSoup(r.content)
    urltemp = str(soup)
    i = 0
    
    while i < len(urltemp):
        if urltemp[i:i+5] == '"url"':
            tempurl = urltemp[i+7:i+200].partition('"')[0]
            if 'listing' in tempurl and 'parts' not in tempurl and 'hardtop' not in tempurl and 'tool' not in tempurl and 'memorabilia' not in tempurl and 'tool kit' not in tempurl and 'inline' not in tempurl and 'removable' not in tempurl:
                if '\\' in tempurl:
                    tempurl = tempurl.replace('\\', '')
                    if tempurl not in urls:
                        urls.append(tempurl.replace('\\', ''))
        i += 1

def get_data(url):

    r = requests.get(url)

    soup = BeautifulSoup(r.content, "html.parser")
    return str(soup)


souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in urls)
display('done')

KeyboardInterrupt: 

In [3]:
souplist = []

#url = 'https://bringatrailer.com/listing/1962-jaguar-xke-roadster-18/'
for i in range(0,len(urls)):
    if i > 10:
        break
    
    r = requests.get(urls[i])
    soup = BeautifulSoup(r.content)

    souplist.append(str(soup))
    
    if i % 5 == 0:
        print("I did 5")



I did 5
I did 5
I did 5


In [34]:
### WRITE DATA TO FILE ###
delimiter = "%newline%"
data = delimiter.join(souplist)
with open("./data/Jaguar_E_Type.html", "w") as file:
    file.write(data)

In [35]:
### READ DATA ###
with open("./data/Jaguar_E_Type.html", "r") as file:
    test_data = file.read()
out = test_data.split(delimiter)

In [41]:
ex = BeautifulSoup(out[2], 'html.parser')
ex.find('title')

<title>1968 Jaguar XKE Series 1.5 4.2 Roadster 4-Speed for sale on BaT Auctions - sold for $95,000 on January 7, 2023 (Lot #95,283) |  Bring a Trailer</title>

In [23]:
filenames = os.listdir("./data")
souplist = []
for fname in filenames:
    with open(f"./data/{fname}", "r") as file:
        data = BeautifulSoup(file, 'html.parser')
        souplist.append(data)

In [23]:
os.getcwd()

'/Users/stephenepstein/version_control/vintage_car_exploration'

In [3]:
url = "https://bringatrailer.com/jaguar/"
urls = []
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")


<!DOCTYPE html>

<html data-pusher="general" lang="en-US" xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://ogp.me/ns#">
<head>
<meta content="True" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Jaguar For Sale - BaT Auctions</title>
<meta content="max-image-preview:large" name="robots">
<link href="//js.stripe.com" rel="dns-prefetch"/>
<link href="//www.google.com" rel="dns-prefetch"/>
<link href="//js.pusher.com" rel="dns-prefetch"/>
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<!-- This site uses the Google Analytics by MonsterInsights plugin v8.7.0 - Using Analytics tracking - https://www.monsterinsights.com/ -->
<script async="" data-cfasync="false" data-wpfc-render="false" src="//www.googletagmanager.com/gtag/js?id=UA-285912-3" type="text/javascript"></script>
<script data-cfasync="false" data-wpfc-render="false" type="text/javascript">
				var mi_version = '8.7.0';
	